# Import modules

In [ ]:
from dataset.speech.convert_mp3_to_wav import convert_mp3_to_wav
from dataset.text.vocabulary import Vocabulary
from dataset.dataset import CustomDataset

# Dataset

In [ ]:
convert_mp3_to_wav("dataset//Vietnamese-Speech-to-Text-datasets//Common-Voice")
convert_mp3_to_wav("dataset//Vietnamese-Speech-to-Text-datasets//ViVOS")

In [ ]:
SAMPLE_RATE = 16000
NUM_SAMPLE = 55000 # Chưa chọn được
TRANSFORMATION = "MFCC"
VOCAB = Vocabulary()

In [ ]:
common_voice_train = CustomDataset(
    audio_directory="dataset//Vietnamese-Speech-to-Text-datasets//Common-Voice//converted voices",
    data_directory="dataset//Vietnamese-Speech-to-Text-datasets//Common-Voice//train",
    target_sample_rate=SAMPLE_RATE,
    num_samples = NUM_SAMPLE,
    transformation=TRANSFORMATION,
    vocab=VOCAB
)

common_voice_dev = CustomDataset(
    audio_directory="dataset//Vietnamese-Speech-to-Text-datasets//Common-Voice//converted voices",
    data_directory="dataset//Vietnamese-Speech-to-Text-datasets//Common-Voice//dev",
    target_sample_rate=SAMPLE_RATE,
    num_samples = NUM_SAMPLE,
    transformation=TRANSFORMATION,
    vocab=VOCAB
)

common_voice_test = CustomDataset(
    audio_directory="dataset//Vietnamese-Speech-to-Text-datasets//Common-Voice//converted voices",
    data_directory="dataset//Vietnamese-Speech-to-Text-datasets//Common-Voice//test",
    target_sample_rate=SAMPLE_RATE,
    num_samples = NUM_SAMPLE,
    transformation=TRANSFORMATION,
    vocab=VOCAB
)

In [ ]:
vivos_train = CustomDataset(
    audio_directory="dataset//Vietnamese-Speech-to-Text-datasets//ViVOS//converted voices",
    data_directory="dataset//Vietnamese-Speech-to-Text-datasets//ViVOS//train.json",
    target_sample_rate=SAMPLE_RATE,
    num_samples = NUM_SAMPLE,
    transformation=TRANSFORMATION,
    vocab=VOCAB
)

vivos_test = CustomDataset(
    audio_directory="dataset//Vietnamese-Speech-to-Text-datasets//ViVOS//converted voices",
    data_directory="dataset//Vietnamese-Speech-to-Text-datasets//ViVOS//test.json",
    target_sample_rate=SAMPLE_RATE,
    num_samples = NUM_SAMPLE,
    transformation=TRANSFORMATION,
    vocab=VOCAB
)

TypeError: CustomDataset.__init__() missing 1 required positional argument: 'vocab'